In [1]:
import numpy as np
import tensorflow as tf


#from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import urllib
import os
import csv
import cv2
import time
from PIL import Image
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

  Using cached tensorflow-2.2.0-cp38-cp38-manylinux2010_x86_64.whl (516.3 MB)
  Using cached tensorboard-2.2.2-py3-none-any.whl (3.0 MB)
  Using cached tensorflow_estimator-2.2.0-py2.py3-none-any.whl (454 kB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0


In [6]:
model_path = '/home/kimsoohyun/00-Research/02-Graph/02-image_detection/04-clickable/dataset/03-models/retinanet/resnet50_csv_49.h5'
model = models.load_model(model_path, backbone_name='resnet50')


AttributeError: module 'tensorflow.python.framework.ops' has no attribute '_TensorLike'

In [ ]:
THRES_SCORE = 0.35
test = [0,0,0,0,0,0,0,0,0,0]
ex = {'filename':['/home/shkim/dataset/test1.jpg',
                  '/home/shkim/dataset/test2.jpg',
                  '/home/shkim/dataset/test3.jpg',
                  '/home/shkim/dataset/test4.jpg',
                  '/home/shkim/dataset/test5.jpg',
                  '/home/shkim/dataset/test6.jpg',
                  '/home/shkim/dataset/test7.jpg',
                  '/home/shkim/dataset/test8.jpg',
                  '/home/shkim/dataset/test9.jpg',
                  '/home/shkim/dataset/test10.jpg'
                 ],'x_min':test,'y_min':test,'x_max':test,'y_max':test,'class_name':['clickable',
                                                                                    'clickable',
                                                                                    'clickable',
                                                                                    'clickable',
                                                                                    'clickable',
                                                                                    'clickable',
                                                                                    'clickable',
                                                                                    'clickable',
                                                                                    'clickable',
                                                                                    'clickable']}
ex_df = pd.DataFrame.from_dict(ex)
def predict(image):
    image = preprocess_image(image.copy())
    image, scale = resize_image(image)
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    boxes /= scale
    return boxes, scores, labels
def draw_detections(image, boxes, scores, labels):
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < THRES_SCORE:
            break
        color = label_color(label)
        b = box.astype(int)
        draw_box(image, b, color=color)
        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(image, b, caption)
def show_detected_objects(image_row):
    img_path = image_row.filename
    image = read_image_bgr(img_path)
    boxes, scores, labels = predict(image)
    print(len(boxes[0]))
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
    true_box = [image_row.x_min, image_row.y_min, image_row.x_max, image_row.y_max]
    draw_box(draw, true_box, color=(255, 0, 0))
    draw_detections(draw, boxes, scores, labels)
    plt.axis('off')
    plt.imshow(draw)
    plt.show()
ex_df